[Link to tutorial](https://juliaai.github.io/DataScienceTutorials.jl/getting-started/stacking/)

In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/Repos/mike_scratch/mlj_tutorial/A-stacking`


┌ Warning: The active manifest file has dependencies that were resolved with a different julia version (1.7.1). Unexpected behavior may occur.
└ @ nothing /Users/mph/Repos/mike_scratch/mlj_tutorial/A-stacking/Manifest.toml:0


Stacking blends the predictions of different regressors or classifiers to gain better performance than naive averaging or majority vote.

For routine stacking tasks, use the `Stack` model ([documentation](https://alan-turing-institute.github.io/MLJ.jl/dev/composing_models/#Model-Stacking)).

Here we will build a two-model stack as an MLJ learning network, which we export as a stand-alone composite model type `MyTwoStack`. The objective here is to (i) explain how stacking works; and (ii) give an advanced demonstration of MLJ's composite model interface.

In [3]:
using MLJ
# using PyPlot
using StableRNGs

In [7]:
Pkg.build("PyCall")

    Building Conda ─→ `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/6cdc8832ba11c7695f494c9d9a1c31e90959ce0f/build.log`


    Building PyCall → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/62f417f6ad727987c755549e9cd88c46578da562/build.log`


Pkg.Types.PkgError: Error building `PyCall`: 
Collecting package metadata (current_repodata.json): ...working... done
Solving environment: ...working... done

# All requested packages already installed.

┌ Info: Using the Python distribution in the Conda package by default.
└ To use a different Python version, set ENV["PYTHON"]="pythoncommand" and re-run Pkg.build("PyCall").
[ Info: Running `conda install -y numpy` in root environment
ERROR: LoadError: Couldn't find libpython; check your PYTHON environment variable.

The python executable we tried was /Users/mph/.julia/conda/3/bin/python (= version 3.10).
Re-building with
    ENV["PYCALL_DEBUG_BUILD"] = "yes"
may provide extra information for why it failed.

Stacktrace:
 [1] error(s::String)
   @ Base ./error.jl:35
 [2] find_libpython(python::String; _dlopen::typeof(Base.Libc.Libdl.dlopen))
   @ Main ~/.julia/packages/PyCall/twYvK/deps/buildutils.jl:72
 [3] find_libpython(python::String)
   @ Main ~/.julia/packages/PyCall/twYvK/deps/bu

In [4]:
linear = (@load LinearRegressor pkg=MLJLinearModels)()
knn = (@load KNNRegressor)()

tree_booster = (@load EvoTreeRegressor)()
forest = (@load RandomForestRegressor pkg=DecisionTree)()
svm = (@load SVMRegressor)()

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/mph/.julia/packages/MLJModels/tMgLW/src/loading.jl:168


import MLJLinearModels ✔

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/mph/.julia/packages/MLJModels/tMgLW/src/loading.jl:168


import NearestNeighborModels ✔
import EvoTrees ✔

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/mph/.julia/packages/MLJModels/tMgLW/src/loading.jl:168


...

ERROR: LoadError: 

Failed to precompile PyCall [438e738f-606a-5dbb-bf0a-cddfbfd45ab0] to /Users/mph/.julia/compiled/v1.8/PyCall/jl_GhXI3F.
Stacktrace:
  [1] error(s::String)
    @ Base ./error.jl:35
  [2] compilecache(pkg::Base.PkgId, path::String, internal_stderr::IO, internal_stdout::IO, keep_loaded_modules::Bool)
    @ Base ./loading.jl:1707
  [3] compilecache
    @ ./loading.jl:1651 [inlined]
  [4] _require(pkg::Base.PkgId)
    @ Base ./loading.jl:1337
  [5] _require_prelocked(uuidkey::Base.PkgId)
    @ Base ./loading.jl:1200
  [6] macro expansion
    @ ./loading.jl:1180 [inlined]
  [7] macro expansion
    @ ./lock.jl:223 [inlined]
  [8] require(into::Module, mod::Symbol)
    @ Base ./loading.jl:1144
  [9] include(mod::Module, _path::String)
    @ Base ./Base.jl:419
 [10] include(x::String)
    @ ScikitLearn ~/.julia/packages/ScikitLearn/ssekP/src/ScikitLearn.jl:10
 [11] top-level scope
    @ ~/.julia/packages/ScikitLearn/ssekP/src/ScikitLearn.jl:12
 [12] include
    @ ./Base.jl:419 [inlined]
 [13] i

ERROR: LoadError: 

Failed to precompile ScikitLearn [3646fa90-6ef7-5e7e-9f22-8aca16db6324] to /Users/mph/.julia/compiled/v1.8/ScikitLearn/jl_JtfO8K.
Stacktrace:
  [1] error(s::String)
    @ Base ./error.jl:35
  [2] compilecache(pkg::Base.PkgId, path::String, internal_stderr::IO, internal_stdout::IO, keep_loaded_modules::Bool)
    @ Base ./loading.jl:1707
  [3] compilecache
    @ ./loading.jl:1651 [inlined]
  [4] _require(pkg::Base.PkgId)
    @ Base ./loading.jl:1337
  [5] _require_prelocked(uuidkey::Base.PkgId)
    @ Base ./loading.jl:1200
  [6] macro expansion
    @ ./loading.jl:1180 [inlined]
  [7] macro expansion
    @ ./lock.jl:223 [inlined]
  [8] require(into::Module, mod::Symbol)
    @ Base ./loading.jl:1144
  [9] include
    @ ./Base.jl:419 [inlined]
 [10] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt64}}, source::String)
    @ Base ./loading.jl:1554
 [11] t

ErrorException: Failed to precompile MLJScikitLearnInterface [5ae90465-5518-4432-b9d2-8a1def2f0cab] to /Users/mph/.julia/compiled/v1.8/MLJScikitLearnInterface/jl_QllIAE.

Warm-up: define a model type to average predictions

We'll start by creating a model type `MyAverageTwo` that averages the predictions of two deterministic regressors. Here's the learning network:

In [ ]:
X = source()
y = source()

model1 = linear
model2 = knn

m1 = machine(model1, X, y)
y1 = predict(m1, X)

m2 = machine(model2, X, y)
y2 = predict(m2, X)

yhat = 0.5*y1 + 0.5*y2

We wrap the learning network in a learning network machine, which specifies what source nodes are, and which node is for prediction. We use a `Deterministic` surrogate model because we're making point predictions.

In [ ]:
mach = machine(Deterministic(), X, y; predict=yhat)

Since we chose not to wrap our source nodes X and y in data, we can't actually fit this.

Here's the macro that "exports" the learning network as a new composite model.

In [ ]:
@from_network mach begin
    mutable struct MyAverageTwo
        regressor1=model1
        regressor2=model2
    end
end

The defaults in the struct must be specified.

In [ ]:
average_two = MyAverageTwo()

Evaluate the average model on the Boston data set.

In [ ]:
function print_performance(model, data...)
    e = evaluate(model, data...;
                 resampling=CV(rng=StableRNG(71), nfolds=8),
                 measure=rms,
                 verbosity=0)
    μ = round(e.measurement[1], sigdigits=5)
    ste = round(std(e.per_fold[1])/sqrt(8), digits=5)
    println("$model = $μ ± $(2*ste)")
end;

X, y = @load_boston

print_performance(linear, X, y)
print_performance(knn, X, y)
print_performance(average_two, X, y)

## Stacking proper

### Helper functions:

Generating folds for generating out-of-sample predictions.

In [ ]:
folds(data, nfolds) = partition(1:nrows(data), (1/nfolds for i in 1:(nfolds-1))...);

In [ ]:
# example
f = folds(1:10, 3)

We can also use the MLJ method `restrict(X, f, i)` that restricts data X to the ith element (fold) of f, and `corestrict(X, f, i)` that restricts to the corresponding fold complement (the concatenation of all but the ith fold).

In [ ]:
# example
corestrict(string.(1:10), f, 2)

In [ ]:
restrict(string.(1:10), f, 2)

In [ ]:
# Choose some test data and some component models (defaults for the composite model)

# <broken PyPlot code goes here>

![some plot](https://juliaai.github.io/DataScienceTutorials.jl/assets/getting-started/stacking/code/output/s1.svg)

In [ ]:
# models to stack
model1 = linear
model2 = knn

In [ ]:
judge = linear

In [ ]:
# Define the data nad training nodes

steps(x) = x < -3/2 ? -1 : (x < 3/2 ? 0 : 1)
x = Float64[-4, -1, 2, -3, 0, 3, -2, 1, 4]
Xraw = (x = x, )
yraw = steps.(x)
idxsort = sortperm(x)
xsort = x[idxsort]
ysort = yraw[idxsort]
step(xsort, ysort, label="truth", where="mid")


X = source(Xraw)
y = source(yraw)

Our first internal node will represent the three folds (vectors of row indices) for creating out-of-sample predictions.

In [ ]:
f = @node folds(X, 3)
# note: `folds` works on a table. But `X` is a _node_ representing a table. So we need to use `@node` in the syntax.

In [ ]:
f()

In [ ]:
# We could also just overload folds to work on nodes using the `node` function.
folds(X::AbstractNode, nfolds) = node(XX->folds(XX, nfolds), X)
f = folds(X, 3)
f()

For `restrict` and `corestrict`, we'll use the overload technique.

In [ ]:
MLJ.restrict(X::AbstractNode, f::AbstractNode, i) = node((XX,ff) -> restrict(XX, ff, i), X, f);
MLJ.corestrict(X::AbstractNode, f::AbstractNode, i) = node((XX,ff) -> corestrict(XX, ff, i), X, f);

In [ ]:
# Now define machines for training model1 on each fold-complement.
m11 = machine(model1, MLJ.corestrict(X, f, 1), MLJ.corestrict(y, f, 1))
m12 = machine(model1, MLJ.corestrict(X, f, 2), MLJ.corestrict(y, f, 2))
m13 = machine(model1, MLJ.corestrict(X, f, 3), MLJ.corestrict(y, f, 3))

In [ ]:
# Define each out-of-sample prediction of model1:
y11 = predict(m11, restrict(X, f, 1));
y12 = predict(m12, restrict(X, f, 2));
y13 = predict(m13, restrict(X, f, 3));

In [ ]:
y1_oos = vcat(y11, y12, y13);

Worth noting that `vcat` and `hcat` don't need overloading to work on nodes.

Let's check our nework so far by fitting and calling y1_oos.

In [ ]:
fit!(y1_oos, verbosity=0)

hcat(x,y1_oos())

Now repeat the procedure for other models.

In [ ]:
m21 = machine(model2, corestrict(X, f, 1), corestrict(y, f, 1))
m22 = machine(model2, corestrict(X, f, 2), corestrict(y, f, 2))
m23 = machine(model2, corestrict(X, f, 3), corestrict(y, f, 3))
y21 = predict(m21, restrict(X, f, 1));
y22 = predict(m22, restrict(X, f, 2));
y23 = predict(m23, restrict(X, f, 3));

In [ ]:
y2_oos = vcat(y21, y22, y23);
fit!(y2_oos, verbosity=0)

hcat(x, y2_oos())

Now we can merge these into the adjudicator's input table and construct the machine for training the adjudicator.

In [ ]:
X_oos = MLJ.table(hcat(y1_oos, y2_oos))
m_judge = machine(judge, X_oos, y)

When we use the stack to make predictions on new data, we will be feeding the adjudicator ordinary predictions of the base learners (rather than out-of-sample predictions). But so far, we have only defined machines to train the base learners on fold complements, not the full data, which we do now:

In [ ]:
m1 = machine(model1, X, y)
m2 = machine(model2, X, y)

## Define nodes still needed for prediction

In [ ]:
y1 = predict(m1, X);
y2 = predict(m2, X);
X_judge = MLJ.table(hcat(y1, y2))
yhat = predict(m_judge, X_judge)

Alright... So `PyCall` isn't working here. Which means I can't finish this.
